In [ ]:
# %% Deep learning - Section 15.146
#    Freezing weights during learning

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [1]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
plt.style.use('default')


In [2]:
# %% Data

# Load data
data = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Split labels from data
labels = data[:,0]
data   = data[:,1:]

# Normalise data (original range is (0,255))
data_norm = data / np.max(data)


In [3]:
# %% Create train and test datasets

# Convert to tensor (float and integers)
data_tensor   = torch.tensor(data_norm).float()
labels_tensor = torch.tensor(labels).long()

# Split data with scikitlearn (10% test data)
train_data,test_data,train_labels,test_labels = train_test_split(data_tensor,labels_tensor,test_size=0.1)

# Convert to PyTorch datasets
train_data = TensorDataset(train_data,train_labels)
test_data  = TensorDataset(test_data,test_labels)

# Convert into DataLoader objects
batch_size   = 32
train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])


In [4]:
# %% Model class

def gen_model():

    class mnist_FFN(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input   = nn.Linear(784,64)
            self.hidden1 = nn.Linear(64,32)
            self.hidden2 = nn.Linear(32,32)
            self.output  = nn.Linear(32,10)

        # Forward propagation
        def forward(self,x):

            x = F.relu(self.input(x))
            x = F.relu(self.hidden1(x))
            x = F.relu(self.hidden2(x))
            x = self.output(x)

            return x


    # Generate model instance
    ANN = mnist_FFN()

    # Loss function
    loss_fun = nn.CrossEntropyLoss()

    # Optimizer (SGD and small lr for illustration purposes)
    optimizer = torch.optim.SGD(ANN.parameters(),lr=0.001)

    return ANN,loss_fun,optimizer


In [ ]:
# %% Inspect requires_grad

ANN = gen_model()[0]
print(ANN.hidden1.weight.requires_grad)


In [ ]:
# %% Freeze one layer

ANN = gen_model()[0]

for p in ANN.named_parameters():
    if 'input' not in p[0]:
        p[1].requires_grad = False

for p in ANN.named_parameters():
    print("requires_grad status in layer %s: %s" %(p[0],p[1].requires_grad))


In [26]:
# %% Function to train the model with some frozen layers

def train_model(ANN,loss_fun,optimizer):

    # Parameters, inizialise vars
    num_epochs = 100

    losses    = []
    train_acc = []
    test_acc  = []

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Switch off g.d. in all but the output layers during the first half epochs
        if epoch_i < (num_epochs/2):
            for p in ANN.named_parameters():
                if 'output' not in p[0]:
                    p[1].requires_grad = False
        else:
            for p in ANN.named_parameters():
                p[1].requires_grad = True

        # Loop over training batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward propagation and loss
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        ANN.train()

    return train_acc,test_acc,losses,ANN


In [28]:
# %% Let's see what happens

# Model instance and fitting
ANN,loss_fun,optimizer = gen_model()
train_acc,test_acc,losses,ANN = train_model(ANN,loss_fun,optimizer)


In [ ]:
# Plotting

phi = (1+np.sqrt(5))/2
plt.figure(figsize=(phi*5,5))

plt.plot(train_acc,label='Train')
plt.plot(test_acc,label='Test')
plt.axvline(x=len(train_acc)/2,color='grey',linestyle=':',linewidth=0.8,label='g.d. switched on')

plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.savefig('figure39_freezing_weights.png')
plt.show()
files.download('figure39_freezing_weights.png')


In [21]:
# %% Exercise 1
#    Switch off all the weights, but leave all the biases switched on. Can the model still learn (at least, better than
#    chance level)? Then do the opposite: let the weights learn but turn off learnign in the biases. How does the model
#    perform now, and what does this tell you about weights vs. biases?

# %% Function to train the model freezing the weights accordingly
def train_model(ANN,loss_fun,optimizer):

    num_epochs = 100

    losses    = []
    train_acc = []
    test_acc  = []

    for epoch_i in range(num_epochs):

        if epoch_i < (num_epochs/2):
            for p in ANN.named_parameters():
                if 'weight' in p[0]:
                    p[1].requires_grad = False
        else:
            for p in ANN.named_parameters():
                p[1].requires_grad = True

        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        ANN.train()

    return train_acc,test_acc,losses,ANN


In [24]:
# %% Exercise 2
#    Freeze only one layer, e.g., layer fc1 (freeze both the weights and biases). Store the accuracy output as a separate
#    variable, so you run the network again without freezing anything. Then plot the accuracies (with and without
#    freezing) on the same graph. How important is fc1 based on this plot?

# %% Function to train the model freezing the weights accordingly
def train_model(ANN,loss_fun,optimizer):

    num_epochs = 100

    losses    = []
    train_acc = []
    test_acc  = []

    for epoch_i in range(num_epochs):

        if epoch_i < (num_epochs/2):
            for p in ANN.named_parameters():
                if 'hidden1' in p[0]:
                    p[1].requires_grad = False
        else:
            for p in ANN.named_parameters():
                p[1].requires_grad = True

        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            batch_loss.append(loss.item())

            matches     = torch.argmax(yHat,axis=1) == y
            matches_num = matches.float()
            accuracy    = 100 * torch.mean(matches_num)
            batch_acc.append(accuracy)

        losses.append( np.mean(batch_loss) )
        train_acc.append( np.mean(batch_acc) )

        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        ANN.train()

    return train_acc,test_acc,losses,ANN


In [31]:
# %% Exercise 2
#    Continue ...

ANN_extra2,loss_fun,optimizer = gen_model()
train_acc_extra2,test_acc_extra2,losses_extra2,ANN_extra2 = train_model(ANN_extra2,loss_fun,optimizer)


In [ ]:
# %% Exercise 2
#    Continue ...

phi = (1+np.sqrt(5))/2
plt.figure(figsize=(phi*5,5))

plt.plot(train_acc,label='Train')
plt.plot(test_acc,label='Test')
plt.plot(train_acc_extra2,'--',color='tab:blue',label='Train hid1 off')
plt.plot(test_acc_extra2,'--',color='tab:orange',label='Test hid1 off')
plt.axvline(x=len(train_acc)/2,color='grey',linestyle=':',linewidth=0.8,label='g.d. switched on')

plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.savefig('figure42_freezing_weights_extra2.png')
plt.show()
files.download('figure42_freezing_weights_extra2.png')
